In [51]:
import numpy as np
from tqdm import tqdm
import random
import gymnasium as gym
# from gym.wrappers import Monitor
import glob
import io
import matplotlib.pyplot as plt
from IPython.display import HTML
from collections import deque

In [52]:
#Setting up the environment
env = gym.make('Taxi-v3', render_mode='ansi')
state, _ = env.reset()

# The state of the environment
print(state)

print("decoded state")
print(list(env.decode(state)))


#The number of states in the environment
print(env.observation_space.n)

#The number of actions in the environment
print(env.action_space.n)

#Take a step in the environment
next_state, reward, done, _, _ = env.step(1)
print("decoded state")
print(list(env.decode(next_state)))

#Render the environment
env.render()

6
decoded state
[0, 0, 1, 2]
500
6
decoded state
[0, 0, 1, 2]


c:\Users\maste\AppData\Local\Programs\Python\Python310\lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.decode to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.decode` for environment variables or `env.get_wrapper_attr('decode')` that will search the reminding wrappers.
  logger.warn(


'+---------+\n|\x1bR\x1b: | : :\x1bG\x1b|\n| : | : : |\n| : : : : |\n| | : | : |\n|\x1bY\x1b| : |B: |\n+---------+\n  (North)\n'

[![Options](https://www.researchgate.net/publication/292208109/figure/fig2/AS:960497125978114@1606011768506/Taxi-problem-and-an-action-hierarchy.gif)]

In [53]:
# actions:  ['down', 'up', 'right', 'left', 'pickup', 'dropoff']
actions = [0, 1, 2, 3, 4, 5]


goal_states = [[0, 0], [0, 4], [4, 0], [4, 3]]

level_1_options = [0, 1]
# highest level options: ['get', 'put']

get_put_options = [0, 1, 2, 3, 4]
# get options: ['pickup/dropff', 'getR', 'getG', 'getY', 'getB']


navigate_options = [0, 1, 2, 3]
# navigate options: ['down', 'up', 'right', 'left']
Q_level_one_options = np.zeros((500, 2)) # Q-values for level options -> get/put

Q_level_two_options = np.zeros((2, 500, len(get_put_options))) # Q-values for level options -> get/put, toR, toG, toY, toB

Q_level_three_options = np.zeros((4, 25, len(navigate_options))) # Q-values for level options -> down, up, right, left


# Softmax function
def softmax(Q, state, tau):
    q_values = Q[state]
    q_values = q_values / tau
    max_q = np.max(q_values)
    e = np.exp(q_values - max_q)
    dist = e / np.sum(e)
    action = np.random.choice(len(dist), p=dist)
    return action

def epsilon_greedy(Q, state, epsilon):
    if random.uniform(0, 1) < epsilon:
        action = np.random.randint(0, 4)
    else:
        action = np.argmax(Q[state])
    return action

In [54]:
def decode_state(env, state):
    return list(env.decode(state))

In [55]:
### HYPERPARAMETERS ###
ALPHA_Q = 0.2
ALPHA_OPTIONS = 0.3
ALPHA_OPTIONS_1 = 0.4
GAMMA = 0.99
TAU = 0.1
DECAY_CONSTANT = 0.995

In [56]:
def execute_level_1_selection(env, state, Q_level_one_options, Q_level_two_options, Q_level_three_options, option, policy, TAU):
    """
    Execute the level 1 selection

    Args:
    env: the environment
    state: the current state
    level_1_options: the level 1 options
    Q_level_two_options: Q-values for level 2 options
    Q_level_three_options: Q-values for level 3 options
    policy: the policy to use

    Returns:
    next_state: the next state
    reward: the reward
    done: is the episode done
    Q_level_two_options: updated Q-values for level 2 options
    Q_level_three_options: updated Q-values for level 3 options
    """
    
    if option < 2:
        """
        Get the Q-values for the level 2 options based on the option chosen
        Initialize the reward bar to 0
        Initialize the counter to 0
        Initialize the current state to the current state
        Get the Q-values for the chosen option
        Update the Q-values for the chosen option based on the reward and the Q-values for the next state
        """
        optdone = False
        optact = 0
        counter = 0
        reward_bar = 0
        chosen_q = Q_level_two_options[option]
        steps = 0
        while not optdone:
            optact = policy(chosen_q, state, TAU)
            next_state, reward, done, [Q_level_two_options, Q_level_three_options], pick_drop_flag, steps_level_2 = execute_level_2_options(env, Q_level_two_options, Q_level_three_options, state, option, optact, policy, TAU)
            reward_bar += reward * GAMMA**counter
            steps += steps_level_2
            chosen_q[state][optact] += ALPHA_OPTIONS * (reward + GAMMA * np.max(chosen_q[next_state]) - chosen_q[state][optact])
            if pick_drop_flag:
                # print("Enters here")
                optdone = True
                Q_level_one_options[state][option] += ALPHA_Q * (reward_bar - Q_level_one_options[state][option] + GAMMA ** counter * np.max(Q_level_one_options[next_state]))
            counter += 1
            state = next_state
        Q_level_two_options[option] = chosen_q
        return next_state, reward_bar, done, [Q_level_one_options, Q_level_two_options, Q_level_three_options], steps
    # else:
    #     next_state, reward, done, _, _ = env.step(option-2)
    #     Q_level_one_options[state][option] += ALPHA_Q * (reward - Q_level_one_options[state][option] + GAMMA * np.max(Q_level_one_options[next_state]))
    #     steps = 1
    #     return next_state, reward, done, [Q_level_one_options, Q_level_two_options, Q_level_three_options], steps


def execute_level_2_options(env, Q_level_two_options, Q_level_three_options, state, option_level_1, option_level_2, policy, TAU):
    """
    Execute the level 2 options

    Args:
    env: the environment
    Q_level_two_options: Q-values for level 2 options
    Q_level_three_options: Q-values for level 3 options 
    state: the current state
    option: the selected option
    level_1_option: the selected level 1 option
    policy: the policy to use

    Returns:
    next_state: the next state
    reward: the reward
    done: is the episode done
    Q_level_two_options: updated Q-values for level 2 options
    Q_level_three_options: updated Q-values for level 3 options
    """
    optdone = False
    optact = 0
    if option_level_2 == 0: # pickup/dropoff
        """
        Pickup or dropoff
        If level 1 option is get, and option is 0, then pickup(4)
        If level 1 option is put, and option is 0, then dropoff(5)    
        """
        if option_level_1 == 0:
            chosen_action = 4
        elif option_level_1 == 1:
            chosen_action = 5
        steps = 1

        """
        Execute the action if it is pickup or dropoff
        Update the Q-values for the level 2 options based on the reward (500, 5)
        """
        next_state, reward, done, _, _ = env.step(chosen_action)
        Q_level_two_options[option_level_1][state][option_level_2] +=  ALPHA_OPTIONS * (reward + GAMMA * np.max(Q_level_two_options[option_level_1][next_state]) - Q_level_two_options[option_level_1][state][option_level_2])
        """
        If the next state is the goal state, then the option is done
        Returns the next state, reward, done, Q-values for level 2 options, Q-values for level 3 options
        """
        # print("Pickup or dropoff")
        return next_state, reward, done, [Q_level_two_options, Q_level_three_options], True, steps
    
    else: # getR, getG, getY, getB
        """
        If the option is not pickup/dropoff, then it is getR, getG, getY, getB
        Get the index of the chosen action
        Initialize the reward bar to 0
        Initialize the counter to 0
        Initialize the current state to the current state
        Based on the chosen action, get the Q-values for the chosen action (25, 4)
        """
        reward_bar = 0
        chosen_action_index = get_put_options.index(option_level_2) - 1
        counter = 0
        current_state = state
        GOAL_REWARD = 0
        chosen_q = Q_level_three_options[chosen_action_index]
        while not optdone:
            """
            While the option is not done, do the following:
            Decode the state
            Get the state value
            Get the action based on the policy and chosen Q-values(25, 4), state value, and tau 
            Execute the action
            Get the next state value
            Update the reward bar
            Update the Q-values for the chosen action based on the reward and the Q-values for the next state
            """
            state_value = 5 * decode_state(env, state)[0] + decode_state(env, state)[1]
            optact = policy(chosen_q, state_value, TAU)
            next_state, reward, done, _, _ = env.step(optact)
            next_state_value = 5 * decode_state(env, next_state)[0] + decode_state(env, next_state)[1]
            reward_bar += reward*GAMMA**counter
            if decode_state(env, next_state)[:2] == goal_states[chosen_action_index]:
                """
                If the next state is the goal state, then the option is done
                Update the Q-values for the level 2 options based on the reward bar and the Q-values for the next state based on the option chosen by level 3 options
                """
                # print("Goal state reached")
                optdone = True
                GOAL_REWARD = 30
                Q_level_two_options[option_level_1][current_state][option_level_2] += ALPHA_OPTIONS * \
                    (reward_bar - Q_level_two_options[option_level_1][current_state][option_level_2] + \
                     GAMMA ** counter * np.max(Q_level_two_options[option_level_1][next_state]))
            chosen_q[state_value][optact] += ALPHA_OPTIONS_1 * (reward + GOAL_REWARD + GAMMA * np.max(chosen_q[next_state_value]) - chosen_q[state_value][optact])
            counter += 1
            state = next_state
        Q_level_three_options[chosen_action_index] = chosen_q
        steps = counter
        print(reward_bar)
        return next_state, reward_bar, done, [Q_level_two_options, Q_level_three_options], False, steps
                
            

In [57]:
#### SMDP Q-Learning
episodes = 2000


def SMDP_Q_Learning(env, Q_level_one_options, Q_level_two_options, Q_level_three_options, policy, TAU, DECAY_CONSTANT, episodes):
    # Iterate over 1000 episodes
    total_rewards = np.zeros((episodes))
    for i in tqdm(range(episodes)):
        state, _ = env.reset()    
        done = False
        # While episode is not over
        episode_steps = 0
        while not done:
            option = softmax(Q_level_one_options, state, TAU)
            # Choose and execute the level 1 option
            next_state, reward, done, value_array, steps = execute_level_1_selection(env, state, Q_level_one_options, Q_level_two_options, Q_level_three_options, option, policy, TAU)
            Q_values_level_1, Q_values_level_2, Q_values_level_3 = value_array
            state = next_state
            total_rewards[i] += reward
            TAU = max(0.01, TAU * DECAY_CONSTANT)
            episode_steps += steps
            # print(episode_steps)

    return Q_values_level_1, Q_values_level_2, Q_values_level_3, total_rewards


In [58]:
Q_level_one_options, Q_level_two_options, Q_level_three_options, total_rewards = SMDP_Q_Learning(env, Q_level_one_options, Q_level_two_options, Q_level_three_options, epsilon_greedy, TAU, DECAY_CONSTANT, episodes)

  0%|          | 0/2000 [00:00<?, ?it/s]

-59.11798255774501
-70.95115056900356
-61.895288189544964
-38.88827604671346
-1.0
-1.0
-2.9701
-3.940399
-1.0
-1.0
-5.8519850599
-1.0
-1.0
-71.52922267268039
-47.44035124744375
-40.70335535985002
-76.4783370759589
-68.83389185508568
-14.854222890512437
-68.83389185508568
-19.836941046095397
-28.94467727277075
-35.08973716315974
-90.66947834789299
-53.877803258190404
-13.994164535871148
-86.19191869125261
-28.226946740172476
-59.93153470484589
-14.854222890512437
-38.88827604671346
-1.0
-1.0
-1.99
-2.9701
-1.0
-1.0
-2.9701
-1.0
-1.0
-10.466174574128356
-52.94133584143496
-60.33221935779743
-51.01097269957946
-1.0
-4.90099501
-1.99
-9.561792499119552
-1.0
-1.0
-10.466174574128356
-46.90944570448864
-55.24767862361889
-61.121608192577256
-50.516134039979256
-1.99
-1.0
-1.0
-1.99
-1.0
-1.0
-16.548623854991238
-25.28279056684035
-1.0
-3.940399
-1.0
-1.0
-51.50086297258367
-9.561792499119552
-1.0
-1.0
-4.90099501
-1.0
-26.029962661171947
-1.0
-1.0
-5.8519850599
-39.499393286246324
-33.771795

  0%|          | 1/2000 [00:01<42:32,  1.28s/it]


-1.0
-5.8519850599
-1.0
-61.895288189544964
-51.985854342857834
-38.27098590577117
-19.02721317787414
-5.8519850599
-1.0
-1.0
-1.0
-73.7287412749771
-20.63857163563444
-1.0
-1.0
-15.705680661607312
-11.361512828387072
-1.0
-1.0
-1.99
-2.9701
-1.0
-1.0
-35.73883979152814
-1.99
-1.0
-1.0
-36.38145139361286
-38.88827604671346
-1.0
-1.0
-50.01629701008006
-50.01629701008006
-63.02703623502729
-47.44035124744375
-22.217864060085315
-62.65357195457302
-40.70335535985002
-1.0
-1.0
-1.99
-1.0
-13.12541872310217
-1.0
-1.99
-1.0
-1.0
-47.44035124744375
-12.247897700103202
-69.75955643309779
-62.65357195457302
-30.35867819504261
-31.744540498961264
-83.78301099889923
-1.0
-1.0
-1.99
-1.99
-1.0
-1.0
-10.466174574128356
-22.217864060085315
-1.0
-1.0
-65.53878166524818
-16.548623854991238
-7.72553055720799
-1.0
-1.0
-2.9701
-1.0
-1.0
-1.99
-1.0
-7.72553055720799
-2.9701
-1.0
-5.8519850599
-1.0
-1.0
-19.02721317787414
-5.8519850599
-1.0
-1.0
-15.705680661607312
-32.42709509397165
-1.0
-1.0
-50.51613

  0%|          | 2/2000 [00:02<32:42,  1.02it/s]

...

  2%|▏         | 31/2000 [00:15<12:34,  2.61it/s]

-74.25153932360506
-66.22455991010975
-1.0
-31.055091413092185
-1.0
-6.793465209301
-1.0
-1.0
-1.99
-1.0
-2.9701
-36.38145139361286
-82.4252698508823
-11.361512828387072
-59.93153470484589
-58.2879120066779
-1.0
-17.383137616441324
-1.0
-2.9701
-1.0
-1.0
-1.99
-1.0
-35.73883979152814
-1.0
-4.90099501
-1.0
-55.24767862361889
-1.0
-3.940399
-1.0
-76.94641816814732
-1.0
-1.99
-38.27098590577117
-1.0
-1.0
-1.99
-59.11798255774501
-1.0
-1.0
-1.99
-69.45409740716948
-1.0
-1.0
-1.99
-33.10282414303193
-1.0
-1.0
-4.90099501
-5.8519850599
-1.0
-1.0
-55.24767862361889
-3.940399
-1.0
-78.29551033271915
-1.0
-3.940399
-1.0
-58.70503288661112
-1.0
-3.940399
-1.0
-63.02703623502729
-1.0
-2.9701
-1.0
-57.44098766113448
-1.0
-2.9701
-1.0
-59.93153470484589
-76.71355370519932
-83.11778434193083
-1.0
-4.90099501
-1.0
-5.8519850599
-1.0
-15.705680661607312
-44.731652283761456
-1.0
-1.0
-1.99
-1.0
-1.99
-1.0
-1.0
-1.99
-40.70335535985002
-1.0
-1.0
-3.940399
-27.501966404214624
-1.0
-1.0
-3.940399
-54.3390

  2%|▏         | 32/2000 [00:15<16:21,  2.01it/s]

-67.22772425621957
-1.0
-1.0
-1.0
-57.86657778452313
-1.0
-1.0
-1.0
-64.48391852949264
-1.0
-1.0
-1.0
-60.33221935779743
-1.0
-1.0
-1.0
-52.94133584143496
-1.0
-1.0
-62.27633530764951
-1.0
-1.0
-1.0
-61.121608192577256
-1.0
-1.0
-58.70503288661112
-57.86657778452313


KeyboardInterrupt: 

In [ ]:
def plot_rewards(rewards):
    avg_rew = []
    std_dev = []
    plt.figure("Reward Plot")
    scores_window = deque(maxlen=300)
    for reward in rewards:
        scores_window.append(reward)
        avg_rew.append(np.mean(scores_window))
        std_dev.append(np.std(scores_window))
    plt.plot(avg_rew)
    plt.fill_between(range(len(avg_rew)), np.subtract(avg_rew, std_dev), np.add(avg_rew, std_dev), color='b', alpha=0.1)
    plt.xlabel('Episodes')
    plt.ylabel('Average Reward')
    plt.ylim(-500, 200)  # Set the Y-axis range

plot_rewards(total_rewards)

In [ ]:
print(np.mean(total_rewards[-100:]))

-149.22032190172203
